In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn; seaborn.set()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import datetime


In [2]:
# importar datos limpiados
#df = pd.read_csv('csv/AirBNB_2009-2020.csv')
df = pd.read_csv('csv/listings_202010L.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'csv/listings_202010L.csv'

In [ ]:
#df.to_csv(r'csv/AirBNB_202010.csv', index = False)

In [ ]:
#df.drop(df.columns[13:20],axis = 1, inplace = True)
df.columns

In [ ]:
df = df[['id', 'host_id', 'host_since','neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'latitude', 'longitude',
       'room_type', 'accommodates', 'bedrooms', 'price', 'minimum_nights',
       'availability_365', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month']]

In [ ]:
df.columns

In [ ]:
# set columna 'id' como index
df.set_index('id', inplace = True)
df = df.rename(columns={"neighbourhood_cleansed": "Barrio","'calculated_host_listings_count'": "listings_host","neighbourhood_group_cleansed": "Distrito"})

In [ ]:
# fill NaNs in columna 'license'
df['license'].fillna('no', inplace = True)
df['bedrooms'].fillna(df['accommodates']/2.3, inplace = True)
#comprobar si hay NaNs
df.isna().sum()

# convertir en formato daytime columna 'first-review'
df['first_review']= pd.to_datetime(df['first_review'])

# añadir columna para el conteo de crecimiento de numero de apartamentos
df = df.sort_values('first_review')

# mover columna 'Date' al principio
date = df.pop('first_review')
df.insert(0, 'date', date)
df.groupby('Distrito').size()

In [ ]:
# Licencias incluyendo alojamientos sin reseña
df_lic = df[['license', 'date']]
df_lic['date']= pd.to_datetime(df_lic['date'])
df_lic.set_index('date', inplace = True)
df_lic = df_lic.resample('YS').nunique()
df_lic = df_lic.reset_index()
df_lic = df_lic.rename(columns={0:'licencias'})
df_lic.set_index('date', inplace = True)
df_lic['total'] = df_lic['license'].cumsum()
df_lic

In [ ]:
# Crecimiento con Host since
df_host = df[['Distrito', 'room_type', 'date', 'host_since']]
df_host['date']= pd.to_datetime(df_host['host_since'])
df_host['aptos_year'] = 1
df_host.set_index('date', inplace = True)
df_host = df_host.resample('YS').sum()
df_host['total'] = df_host['aptos_year'].cumsum()
df_host

In [ ]:
# Crear df con todas las entradas (al otro se quitan aptos sin reseña)
corona = df[['Distrito', 'room_type', 'date', 'host_since']]
corona = corona[corona['Distrito'] == 'Centro']
corona['aptos_year'] = 1
corona['date']= pd.to_datetime(corona['date'])
corona.set_index('date', inplace = True)
corona = corona.resample('MS').sum()
corona['total'] = corona['aptos_year'].cumsum()
corona

In [ ]:
# Crear df con todas las entradas (al otro se quitan aptos sin reseña)
centro = df[['Distrito', 'room_type', 'date', 'host_since']]
centro = centro[centro['Distrito'] == 'Centro']
centro['aptos_year'] = 1
centro['date']= pd.to_datetime(centro['date'])
centro.set_index('date', inplace = True)
centro = centro.resample('QS').sum()
centro['total'] = centro['aptos_year'].cumsum()
centro

In [ ]:
# Crear df con todas las entradas (al otro se quitan aptos sin reseña)
df_years = df[['Distrito', 'room_type', 'date', 'host_since']]
df_years['aptos_year'] = 1
df_years['date']= pd.to_datetime(df_years['date'])
df_years.set_index('date', inplace = True)
df_years = df_years.resample('YS').sum()
df_years['total'] = df_years['aptos_year'].cumsum()
df_years

In [ ]:
# eliminar filas con NaNs (5.200 sin reseñas)
df = df.dropna()

df['total'] = 1
df['acum'] = df['total'].cumsum()

In [ ]:
# Crear columna para traducir dias a meses como en el Dataframe distritos (luego se hace de una manera más elegante)
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df["newdate"] = df["month"].astype(str) + '/01/' + df["year"].astype(str)
df['newdate']= pd.to_datetime(df['newdate'])
df

In [ ]:
# Crear Dataframe con 10 años de datos de 3 distritos + Madrid
df_dis = pd.read_csv(r'csv/Distritos_actual.csv', sep = ';')
df_dis

In [ ]:
pib = pd.read_csv(r'csv/PIB.csv', sep = ';')
pib

In [ ]:
C_Var = pd.read_csv(r'csv/Var_Centro.csv', sep = ';', decimal = ',')

In [ ]:
C_Var.to_csv(r'csv/C_Var.csv')


In [ ]:
# Set Fecha como index
df_dis['Date']= pd.to_datetime(df_dis['Mes'])
df_dis['Date'] = pd.DatetimeIndex(df_dis['Date'])
df_dis.set_index('Date', inplace = True)
df_dis.drop('Mes', axis = 1, inplace = True)

In [ ]:
#Crear series de datos distritos para cruzarlos del AirBNB a distritos
mask = df['Distrito'] == 'Centro'
df_C = df[mask]
df_C = df_C.groupby('newdate').size()

mask = df['Distrito'] == 'Ciudad Lineal'
df_CL = df[mask]
df_CL = df_CL.groupby('newdate').size()

mask = df['Distrito'] == 'Villa de Vallecas'
df_V = df[mask]
df_V = df_V.groupby('newdate').size()

df_C.name = 'AirBNB_Centro'
df_CL.name = 'AirBNB_CiudadL'
df_V.name = 'AirBNB_Vallecas'

# Añadir info AirBNB a distritos
df_dis = df_dis.join(df_C)
df_dis = df_dis.join(df_CL)
df_dis = df_dis.join(df_V)

In [ ]:
df_dis['AirBNB_Centro'] = df_dis['AirBNB_Centro'].fillna(0)
df_dis['AirBNB_CiudadL'] = df_dis['AirBNB_CiudadL'].fillna(0)
df_dis['AirBNB_Vallecas'] = df_dis['AirBNB_Vallecas'].fillna(0)

In [ ]:
df_dis['CAM PIB M'] = df_dis['CAM PIB M'].astype(int)
df_dis['CAM Var PIB'] = df_dis['CAM Var PIB'].round(3)
df_dis.to_csv(r'csv/Distritos.csv')

In [ ]:
# Elegir solo room type 'entire' para conteo de ratio AirBNB/viviendas
df_total = pd.read_csv('csv/listings_202008.csv')
df_total = df_total.rename(columns={"neighbourhood_cleansed": "Barrio","host_total_listings_count": "listings_host","neighbourhood_group_cleansed": "Distrito"})
mask = df_total['room_type'] == 'Entire home/apt'
ratio = df_total[mask].groupby(['Distrito']).size()

In [ ]:
# Definición de colores
color_discrete_sequence=["#1F6768", "#7DC2BF","#EE2737", "#FBD872" ]
color_discrete_sequence2=["#1F6768", "#7DC2BF","#EE2737", "#FBD872","#aeaeae", '#c3dbdb']
color_continuous_scale=["#EDEBDC", "#FBD872", "#EE2737"]
color_continuous_scale2=["#EE2737",'#ECEDF4', "#FBD872"]

In [ ]:
#centro = centro.acum()

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_years.index, y=df_years.total,
                    mode='lines',
                    name='Apartamentos activos',
                         marker_color='#EE2737'))

fig.add_trace(go.Scatter(x=df_host.index, y=df_host.total,
                    mode='lines',
                    name='Apartamentos/host',
                        marker_color='#1F6768'))

fig.add_trace(go.Scatter(x=df_lic.index, y=df_lic.total,
                    mode='lines',
                    name='Licencias',
                        marker_color='#FBD872'))

fig.add_trace(go.Bar(x=df_host.index, y=df_host.aptos_year, name='Apartamentos/año', marker_color='#EE2737 '))
fig.add_trace(go.Bar(x=df_years.index, y=df_years.aptos_year, name='Apartamentos/año/host', marker_color='#1F6768'))
fig.update_yaxes(showgrid=True, tickformat = '.')
fig.update_xaxes(dtick = 'M12')
fig.update_layout(template="simple_white",
                  width=900,
                  height=500, 
                  legend=dict(
                      yanchor="top",
                      y=0.99,
                      xanchor="left",
                      x=0.01))
fig.show()
fig.write_html(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\Aptos_Madrid.html")

In [ ]:
violin = pd.read_csv('csv/listings_202008.csv')
violin['count'] = 1
data = [violin['host_id'],violin['count']]
violin = pd.concat(data, axis=1)
violin = violin.groupby(['host_id']).sum()
violin = violin.reset_index()
violin

In [ ]:
df_donut = pd.read_csv('csv/listings_202008.csv')
df_donut = df_donut[['host_id','host_name','first_review','calculated_host_listings_count']]
df_donut = df_donut.dropna()
df_donut.sort_values(by = 'calculated_host_listings_count', ascending = False)
values = df_donut.groupby('calculated_host_listings_count').nunique()

In [ ]:
values = values.drop(['host_name', 'calculated_host_listings_count'], axis = 1)

In [ ]:
values['bins'] = values.index * values['host_id']
values['bins2'] = values.index
values['bins2'] = values['bins2'].where(values['bins2'] <= 5, '6-100')
values['bins2'] = values['bins2'].where(values.index <= 100, '>100')

In [ ]:
values = values.groupby('bins2').sum()
values

In [ ]:
fig = px.pie(values, 
             values='bins',
             names=values.index,
             hole=.3, 
             color_discrete_sequence=color_discrete_sequence2, 
             width=800, 
             height=400)
fig.update_layout(template="simple_white",
                  legend=dict(y=0.01,
                              x=0.01))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5))
fig.show()
fig.write_image(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\Aptos_bins.png")

In [ ]:
fig = px.pie(values, 
             values='host_id', 
             names=values.index, 
             hole=.3, 
             color_discrete_sequence=color_discrete_sequence2, 
             width=800, 
             height=400)
fig.update_layout(template="simple_white",
                  legend=dict(y=0.01,
                              x=0.01))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5))
fig.show()
fig.write_image(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\Aptos_perHost.png")

#join df_original con df_aptos_host

In [ ]:
fig = px.violin(violin, y="count", box=True, # draw box plot inside the violin
                points='all',
                color_discrete_sequence=color_discrete_sequence,
                template="simple_white"
               )
fig.show()
fig.write_html(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\violin.html")

In [ ]:
tipos = df['room_type'].value_counts()
tipos = tipos.reset_index()
tipos.rename(columns={'index': "tipos", 'room_type': "numero"}, inplace = True)

In [ ]:

fig = go.Figure([go.Bar(x=tipos['tipos'], y=tipos['numero'], marker_color=color_discrete_sequence)])
fig.update_layout(template="simple_white",
                  width=700,
                  height=500)
fig.update_yaxes(tickformat=".")
fig.update_traces(texttemplate=['9.452','5.174', '208','137'], textposition='outside')
fig.show()

In [ ]:
x = corona1.index
co = corona1['aptos_year']
color_discrete_sequence_co=["#1F6768", "#7DC2BF","#EE2737", "#FBD872","#aeaeae", '#c3dbdb',"#1F6768", "#7DC2BF","#EE2737"]

fig = go.Figure([go.Bar(x=x, y=co, marker_color=color_discrete_sequence_co)])
fig.update_layout(template="simple_white",
                  width=1200,
                  height=600)
fig.update_yaxes(tickformat=".")
fig.update_traces(texttemplate=['171','170', '102','6','12','38','109','76','96'], textposition='outside')
fig.show()

In [ ]:
mapbox_access_token = "pk.eyJ1Ijoia2Vyc3RpbmsiLCJhIjoiY2tnbGJ1enN6MGZ0ZTJ0cGNrbzJid3g4NSJ9.cN4FFIyy2xOcdjGHEUwzFQ"

fig = px.scatter_mapbox(df,
                        lat="latitude",
                        lon="longitude",
                        color = "room_type",
                        animation_frame="year",
                        opacity = 0.8,
                        color_discrete_map={
                "Private room": "#7DC2BF",
                "Entire home/apt": "#EE2737"},
                        #color_discrete_sequence=color_discrete_sequence2,
                        zoom=11)
fig.update_layout(mapbox=dict(accesstoken=mapbox_access_token), mapbox_style = 'dark', width = 1300, height = 600)

fig.show();
fig.write_html(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\DensidadAirBNB2.html")

In [ ]:
mapbox_access_token = "pk.eyJ1Ijoia2Vyc3RpbmsiLCJhIjoiY2tnbGJ1enN6MGZ0ZTJ0cGNrbzJid3g4NSJ9.cN4FFIyy2xOcdjGHEUwzFQ"

fig = ff.create_hexbin_mapbox(df,
                              lat="latitude",
                              lon="longitude",
                              #color = "total",
                              labels={"color": "Nº Apartamentos"},
                              #color_continuous_scale="inferno",
                              color_continuous_scale=color_continuous_scale,
                              nx_hexagon=100,
                              #animation_frame="Year",
                              #agg_func=np.cumsum,
                              #range_color=[50,250],
                              opacity = 0.7,
                              min_count=1,
                              zoom=13,
                              center=dict(lat=40.417,lon=-3.7),)
fig.update_layout(
    mapbox=dict(accesstoken=mapbox_access_token),width = 1200, height = 600)
fig.show() 
fig.write_html(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\Map_hexbin.html")

In [ ]:
#"#789DB7", "#FFB838","#486D87", "#aeaeae"
fig =plt.figure(figsize=(18, 6))
plt.style.use('seaborn-whitegrid')
ax1 = df_dis["C Var"].resample('QS').mean().plot(color = '#EE2737')
ax2 = df_dis["CL Var"].resample('QS').mean().plot(color = '#7DC2BF')
ax3 = df_dis["V Var %"].resample('QS').mean().plot(color = '#FBD872')
ax4 = df_dis["M Var"].resample('QS').mean().plot(color = '#333333')
ax5 = df_dis["CAM Var PIB"].plot(color = '#333333',style = ':')
plt.ylabel('Variación/trimestre precio alquiler/PIB', color='#333333')
plt.legend(['Centro','Ciudad Lineal','Vallecas','Madrid','PIB Madrid'], loc = 'upper left');
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\var_precios.png")

In [ ]:
df_dis['C Padron'] = round(df_dis['C Padron']/3,0)

In [ ]:
oferta

In [ ]:
x = oferta.index
C = oferta['Centro']
M = oferta['Ciudad de Madrid']
fig =plt.figure(figsize=(12,8))
ax1 = C.plot(color = '#1F6768',  marker = 'o')
ax2 = M.plot(color = '#EE2737', marker = 'o')
#ax2.grid(False)

plt.ylabel('Precio alquiler/m2 en €', color='#333333')
#plt.grid(axis = 'x')
#plt.ylim(bottom=6)
#plt.xlim([datetime.date(2010,5,1), datetime.date(2020,9,1)])
plt.style.use('seaborn-whitegrid')
plt.legend(['Centro','Madrid'], loc = 'upper right');
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")
#fig.write_image(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")


In [ ]:
corona1 = corona[-10:-1]
corona1

In [ ]:
x = corona1.index
co = corona1['aptos_year']


fig =plt.figure(figsize=(15,10))
ax1 = co.plot(color = '#1F6768',  marker = 'o')
#ax2.grid(False)

plt.ylabel('Precio alquiler/m2 en €', color='#333333')
#plt.grid(axis = 'x')
#plt.ylim(bottom=6)
#plt.xlim([datetime.date(2010,5,1), datetime.date(2020,9,1)])
plt.style.use('seaborn-whitegrid')
plt.legend(['Centro','Madrid'], loc = 'upper right');
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")
#fig.write_image(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")


In [ ]:
x = oferta.index
C = oferta['Oferta viviendas']

fig =plt.figure(figsize=(12,8))
ax = C.plot(color = '#548582',  marker = 'o').fill_between(x,0,C, color='#7DC2BF', alpha = 0.8)

plt.ylabel('Variación oferta vivienda alquiler Idealista', color='#333333')

plt.style.use('seaborn-whitegrid')
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\oferta_idealista.png")

In [ ]:
fig =plt.figure(figsize=(15,5))
ax1 = df_dis["C Precio m2"].plot(color = '#1F6768').fill_between(df_dis.index,0,df_dis["C Precio m2"], color='#1F6768', alpha = 0.7)
ax2 = df_dis["CL Precio"].plot(color = '#548582').fill_between(df_dis.index,0,df_dis["CL Precio"], color='#7DC2BF', alpha = 0.8)
ax3 = df_dis["V Precio m2"].plot(color = '#EE2737').fill_between(df_dis.index,0,df_dis["V Precio m2"], color='#ed6b75', alpha = 0.9)
ax4 = df_dis["M Precio m2"].plot(color = 'white',linewidth=3, style = '--')
plt.ylabel('Precio alquiler/m2 en €', color='#333333')
#plt.grid(axis = 'x')
plt.ylim(bottom=6)
plt.xlim([datetime.date(2010,5,1), datetime.date(2020,9,1)])
plt.style.use('seaborn')
plt.legend(['Centro','Ciudad Lineal','Vallecas','Madrid'], loc = 'upper left');
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")
#fig.write_image(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_alqiileres.png")

In [ ]:
oferta = pd.read_csv('csv/Oferta_alquiler.csv', encoding = 'Latin', sep = ';', decimal=",")
oferta.rename(columns={"Unnamed: 0": "Date"}, inplace = True)
oferta['Date'] = pd.DatetimeIndex(oferta['Date'])
oferta.set_index('Date', inplace = True)
oferta

In [ ]:
x = oferta['Centro']
C = oferta['Centro']
M = oferta['Ciudad de Madrid']

In [ ]:

fig, ax1 = plt.subplots(figsize=(13, 7))
x = df_dis.index
C = df_dis["C Precio m2"].resample('Q').mean()
D = centro['total']

color = '#FBD872'
ax1.set_ylabel('Precio m2 en Centro', color=color, fontsize=18)
ax1.plot(C, color=color,  marker="o")
#.fill_between(df_dis.index,0,df_dis["C Precio m2"], color='#789DB7')
ax1.tick_params(axis='y', labelcolor=color, labelsize=12)
ax1.tick_params(axis='x', labelcolor=color, labelsize=12)

#ax1.grid(False)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = '#EE2737'
ax2.set_ylabel('Aptos nuevos Centro', color=color, fontsize=18)  # we already handled the x-label with ax1
ax2.plot(D, color=color, marker="o")
ax2.tick_params(axis='y', labelcolor=color, labelsize=12)
ax2.grid(False)


#plt.style.use("dark_background")
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show();
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\precio_vs_AirBNB.png");

In [ ]:
fig, ax1 = plt.subplots(figsize=(13, 7))
x = df_dis.index
C = df_dis["C Precio m2"].resample('Q').mean()
D = centro['total']
E = df_dis['C Padron'].resample('Q').mean()

color = 'w'
ax1.set_ylabel('Saldo Padrón', color=color, fontsize = 16)
ax1.plot(E, color=color, marker="o")
#.fill_between(df_dis.index,0,df_dis["C Precio m2"], color='#789DB7')
ax1.tick_params(axis='y', labelcolor=color, labelsize=12)
ax1.tick_params(axis='x', labelcolor=color, labelsize=12)

ax1.grid(False)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = '#EE2737'
ax2.set_ylabel('Aptos nuevos Centro', color=color)  # we already handled the x-label with ax1
ax2.plot(D, color=color, marker="o")
ax2.tick_params(axis='y', labelcolor=color, labelsize=12)
ax2.yaxis.label.set_size(16)
ax2.grid(False)


#plt.style.use("dark_background")
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show();
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\padron_vs_AirBNB.png");

In [ ]:
fig, ax1 = plt.subplots(figsize=(13, 7))
x = df_dis.index
D = centro['total']
F = df_dis['CAM PIB M'].resample('Q').mean()

color = '#7DC2BF'
ax1.set_ylabel('PIB CAM', color=color, fontsize = 16)
ax1.plot(F, color=color , marker="o")
#.fill_between(df_dis.index,0,df_dis["C Precio m2"], color='#789DB7')
ax1.tick_params(axis='y', labelcolor=color, labelsize=12)
ax1.tick_params(axis='x', labelcolor=color, labelsize=12)

ax1.grid(False)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = '#EE2737'
ax2.set_ylabel('Aptos nuevos Centro', color=color)  # we already handled the x-label with ax1
ax2.plot(D, color=color, marker="o")
ax2.tick_params(axis='y', labelcolor=color, labelsize=12)
ax2.yaxis.label.set_size(16)
#ax2.grid(False)

#plt.style.use("dark_background")
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show();
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\PIB_vs_AirBNB.png");

In [ ]:
viviendas = pd.read_csv('csv/Viviendas.csv', encoding = 'Latin', sep = ';', decimal=",")
viviendas['Variación anual'] = viviendas['Variación anual']*100
viviendas

In [ ]:
# Importar datos de número de viviendas

#viviendas.drop('Unnamed: 2', axis = 1, inplace = True)
viviendas.rename(columns={"Unnamed: 0": "Distritos"}, inplace = True)
viviendas.set_index('Distritos', inplace = True)

# Join numero viviendas y datos AirBNB
viviendas.name = 'viviendas'
ratio.name = 'AirBNB'
df_ratio = viviendas.join(ratio)
df_ratio['ratioAirBNB'] = df_ratio['AirBNB']/df_ratio['Viviendas']
df_ratio['Distritos'] = df_ratio.index

In [ ]:
sns.displot(df_ratio['ratioAirBNB'], kde=True, height=6, aspect=6/6)
plt.xlabel('Ratio aptos turísticos/vivienda')
plt.ylabel('Número barrios');
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\displot.png")

In [ ]:
df_ratio= df_ratio.sort_values(by = 'ratioAirBNB', ascending = False)
df_ratio.describe()

In [ ]:
fig =plt.figure(figsize=(8, 6))
ax = sns.barplot(x=df_ratio.index, y="ratioAirBNB", data=df_ratio,
                 palette="Blues_r")
plt.ylabel('Ratio aptos turísticos/vivienda');
plt.xticks(rotation=90);
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\ratio.png")

In [ ]:
df_centro = df_dis[['C Precio m2', 'C Padron','C Var','CAM Var PIB','CAM PIB M', 'AirBNB_Centro']]


In [ ]:
df_centro.rename({'C Precio m2':'Precio Alquiler', 'C Padron':'Padron', 'AirBNB_Centro':'AirBNB'}, axis = 1, inplace = True)
#df_centro.drop(['C Var', 'CAM Var PIB'], axis = 1, inplace = True)

In [ ]:
df_corr = df_centro.corr()

In [ ]:
plt.figure(figsize=(10,8.5))
sns.heatmap(df_corr, vmin=-1, vmax=1, 
            annot=True,
            linewidths=1,
            cmap = sns.diverging_palette(12, 220, as_cmap=True))
#sns.set(font_scale=1.2)
#plt.style.use("dark_background")
plt.show();
plt.savefig(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\HEATMAP.png")

In [ ]:
from urllib.request import urlopen
import json
import geojson
import pandas as pd
import plotly.express as px
with open('DISTRITOS.geojson') as f:
    madrid = geojson.load(f)

fig = px.choropleth_mapbox(df_ratio, 
                           geojson=madrid, 
                           locations="Distritos", 
                           featureidkey='properties.name',
                           color='Variación anual',
                           zoom=10,
                           center=dict(lat=40.417,lon=-3.7),
                           #center={"lat": 45.5517, "lon": -73.7073}
                           opacity = 0.8,
                           color_continuous_scale=color_continuous_scale2,
                           color_continuous_midpoint=0)

mapbox_access_token = "pk.eyJ1Ijoia2Vyc3RpbmsiLCJhIjoiY2tnbGJ1enN6MGZ0ZTJ0cGNrbzJid3g4NSJ9.cN4FFIyy2xOcdjGHEUwzFQ"

fig.update_layout(
    mapbox=dict(accesstoken=mapbox_access_token))
fig.show() 
fig.write_html(r"C:\Users\Kerstin\Data Science\Proyectos\Proyecto_Analitica\plots\Map_alquiler2020.html")